In [46]:
import copy
import math
import numpy as np

'''
    Pure Python/Numpy implementation of the Nelder-Mead algorithm.
    Reference: https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method
'''

def nelder_meadus(f, x_start, bounds,
                step=0.1, no_improve_thr=1e-3,
                no_improv_break=10, max_iter=500,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    '''
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
        @param x_start (numpy array): initial position
        @param bounds (list): 3-dimensional bound for each parameter, as a list of 3 tuples
        @param step (float): look-around radius in initial step
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)
        return: tuple (best parameter array, best score)
    '''

    # init
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        x = np.clip(x, bounds[i][0], bounds[i][1])
        score = f(x)
        res.append([x, score])

    # simplex iter
    iters = 0
    fev = 1
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]

        # break after max_iter
        if max_iter and iters >= max_iter:
#             return np.clip(res[0][0], bounds[0][0], bounds[0][1]), res[0][1]
              return res[0], iters, fev
#         iters += 1

        # break after no_improv_break iterations with no improvement
        print('...best so far:', best)
        print('function evaluations:', fev)

#         if best < prev_best - no_improve_thr:
#             no_improv = 0
#             prev_best = best
#         else:
#             no_improv += 1

#         if no_improv >= no_improv_break:
#             return np.clip(res[0][0], bounds[0][0], bounds[0][1]), res[0][1]
        
        
        iters += 1

        # centroid
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        

                    # reflection
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        fev += 1
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # expansion
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            fev += 1
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # contraction
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        fev += 1
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # reduction
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            fev += 1
            nres.append([redx, score])
        res = nres

        # check if the result is within the bounds
        for i in range(len(res)):
            for j in range(len(x_start)):
                if res[i][0][j] < bounds[j][0]:
                    res[i][0][j] = bounds[j][0]
                elif res[i][0][j] > bounds[j][1]:
                    res[i][0][j] = bounds[j][1]

#         # stopping criterion
#         if rscore < 1e-6:
#             return res[0], iters, fev
#         print(fev)
        return res[0], iters, fev


In [47]:
import math
import numpy as np

# Define the function to optimize
def f(x):
    return math.sin(x[0]) * math.cos(x[1]) * (1. / (abs(x[2]) + 1))

# Define the initial position and bounds
x_start = np.array([0., 0., 0.])
bounds = [(0, 10), (-5, 5), (-1, 1)]

# Run the optimization using Nelder-Mead method
best_score, iters, fev = nelder_meadus(f, x_start, bounds=bounds)
# nelder_meadus(f, x_start, bounds=bounds)

# Print the results
# print("Best parameters:", best_param)
print("Best score:", best_score)
print("Iterations:", iters)
print("Function evaluations:", fev)


...best so far: 0.0
function evaluations: 1
...best so far: -0.17970619241280353
function evaluations: 3
...best so far: -0.17970619241280353
function evaluations: 4
...best so far: -0.30657463012029224
function evaluations: 6
...best so far: -0.5263352673542919
function evaluations: 8
...best so far: -0.5263352673542919
function evaluations: 9
...best so far: -0.7526372727014512
function evaluations: 11
...best so far: -0.7526372727014512
function evaluations: 12
...best so far: -0.7526372727014512
function evaluations: 13
...best so far: -0.7526372727014512
function evaluations: 15
...best so far: -0.7526372727014512
function evaluations: 17
...best so far: -0.8877444448223267
function evaluations: 19
...best so far: -0.8877444448223267
function evaluations: 20
...best so far: -0.8877444448223267
function evaluations: 21
...best so far: -0.8877444448223267
function evaluations: 23
...best so far: -0.9039290159484077
function evaluations: 25
...best so far: -0.9053729782222878
functio